In [ ]:
import tensorflow as tf

from tensorflow.keras.layers import Dense, Flatten, Conv2D
from tensorflow.keras import Model
EPOCHS = 10

mnist = tf.keras.datasets.mnist
fashion_mnist = tf.keras.datasets.fashion_mnist

(x_train, y_train), (x_test, y_test) = fashion_mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

# Add a channels dimension e.g. (60000, 28, 28) => (60000, 28, 28, 1)
x_train = x_train[..., tf.newaxis]
x_test = x_test[..., tf.newaxis]

train_ds = tf.data.Dataset.from_tensor_slices(
    (x_train, y_train)).shuffle(10000).batch(32)
test_ds = tf.data.Dataset.from_tensor_slices((x_test, y_test)).batch(32)

class CustomModel(Model):
  def __init__(self):
    super(CustomModel, self).__init__()
    self.conv1 = Conv2D(32, 3, activation='relu')
    self.flatten = Flatten()
    self.d1 = Dense(128, activation='relu')
    self.d2 = Dense(10, activation='softmax')

  def call(self, x):
    x = self.conv1(x)
    x = self.flatten(x)
    x = self.d1(x)
    return self.d2(x)

model = CustomModel()

loss_object = tf.keras.losses.SparseCategoricalCrossentropy()
optimizer = tf.keras.optimizers.Adam()

train_loss = tf.keras.metrics.Mean(name='train_loss')
train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='train_accuracy')

test_loss = tf.keras.metrics.Mean(name='test_loss')
test_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='test_accuracy')

@tf.function
def train_step(images, labels):
  with tf.GradientTape() as tape:
    predictions = model(images)
    loss = loss_object(labels, predictions)
  gradients = tape.gradient(loss, model.trainable_variables)
  optimizer.apply_gradients(zip(gradients, model.trainable_variables))

  train_loss(loss)
  train_accuracy(labels, predictions)

@tf.function
def test_step(images, labels):
  predictions = model(images)
  t_loss = loss_object(labels, predictions)

  test_loss(t_loss)
  test_accuracy(labels, predictions)

for epoch in range(EPOCHS):
  for images, labels in train_ds:
    train_step(images, labels)

  for test_images, test_labels in test_ds:
    test_step(test_images, test_labels)

  template = 'Epoch {}, Loss: {}, Accuracy: {}, Test Loss: {}, Test Accuracy: {}'
  print (template.format(epoch+1,
                         train_loss.result(),
                         train_accuracy.result()*100,
                         test_loss.result(),
                         test_accuracy.result()*100))

# Save the weights
model.save('fashion_mnist')

4422102/4422102 [==============================] - 1s 0us/step
Epoch 1, Loss: 0.3704822361469269, Accuracy: 86.77666473388672, Test Loss: 0.2953467071056366, Test Accuracy: 89.20999908447266
Epoch 2, Loss: 0.3058874011039734, Accuracy: 88.94083404541016, Test Loss: 0.2877168357372284, Test Accuracy: 89.54499816894531
Epoch 3, Loss: 0.2658371925354004, Accuracy: 90.31999969482422, Test Loss: 0.2810825705528259, Test Accuracy: 89.91666412353516
Epoch 4, Loss: 0.23525060713291168, Accuracy: 91.41791534423828, Test Loss: 0.2832569479942322, Test Accuracy: 90.07250213623047
Epoch 5, Loss: 0.21008475124835968, Accuracy: 92.32666778564453, Test Loss: 0.29004859924316406, Test Accuracy: 90.16400146484375
Epoch 6, Loss: 0.1887846738100052, Accuracy: 93.11194610595703, Test Loss: 0.2985830307006836, Test Accuracy: 90.29000091552734
Epoch 7, Loss: 0.17041857540607452, Accuracy: 93.79618835449219, Test Loss: 0.3087095618247986, Test Accuracy: 90.40142822265625
Epoch 8, Loss: 0.15502354502677917, A